In [1]:
import pandas as pd
import requests
import numpy as np
from pandas import json_normalize
from pathlib import Path
from api_key import api_key

In [2]:
# Station codes as strings
station_code_list = ["BG001"]

In [3]:
# Headers with API key
headers = {
    "api_key": api_key
}

In [ ]:
# Initialize an empty list to store the data for all stations
all_stations_data = []

# Number of iterations
num_iterations = 600

# Iterate through each station
for station in station_code_list:
    # Iterate over the specified number of times
    for i in range(num_iterations):
        # Parameters for the specific station
        url = f"https://api.dpird.wa.gov.au/v2/weather/stations/{station}/data?"
        params = {
            "startDateTime": "2023-01-01T00:00:00Z",
            "endDateTime": "2023-07-01T00:00:00Z",
            "offset": i * 1000,  # Use the current iteration index as the offset
            "limit": 1000,
        }

        # Make the request for each station
        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        # Check if the 'collection' list is non-empty
        if "collection" in data:
            # Extract relevant information for each summary
            for summary in data["collection"]:
                try:
                    station_info = {
                        "dateTime": summary["dateTime"],
                        "airTemperature": summary["airTemperature"],
                        "relativeHumidity": summary["relativeHumidity"],
                        "soilTemperature": summary["soilTemperature"],
                        "solarIrradiance": summary["solarIrradiance"],
                        "rainfall": summary["rainfall"],
                        "dewPoint": summary["dewPoint"],
                        "deltaT": summary["deltaT"],
                        "wetBulb": summary["wetBulb"],
                        "wind_avg_speed": summary["wind"][0]["avg"]["speed"],
                        "wind_direction": summary["wind"][0]["avg"]["direction"]["compassPoint"],
                        "barometricPressure": summary["barometricPressure"],
                        "batteryVoltage": summary["batteryVoltage"],
                    }
                except KeyError as e:
                    # Handle specific exception (e.g., key not found in the response)
                    # Use a default station_info with all values as np.nan
                    station_info = {
                        "dateTime": np.nan,
                        "airTemperature": np.nan,
                        "relativeHumidity": np.nan,
                        "soilTemperature": np.nan,
                        "solarIrradiance": np.nan,
                        "rainfall": np.nan,
                        "dewPoint": np.nan,
                        "deltaT": np.nan,
                        "wetBulb": np.nan,
                        "wind_avg_speed": np.nan,
                        "wind_direction": np.nan,
                        "barometricPressure": np.nan,
                        "batteryVoltage": np.nan,
                    }

                # Append the information for the current station and summary to the list
                all_stations_data.append(station_info)

In [ ]:
# Assuming all_stations_data is a list of dictionaries
all_stations = pd.DataFrame(all_stations_data)

# Display dataframe
all_stations

In [ ]:
all_stations.to_csv("../Resources/raindata.csv", index=False)